# (OpenCV - Chap6) 화소(pixel)처리
> 영상 화소의 접근과 화소 밝기 변환
- toc: true
- branch: master
- badges: false
- comments: true
- author: pinkocto
- categories: [python]

## 화소의 개념

화소란 화면(영상)을 구성하는 가장 기본이 되는 단위를 말한다. 일반적으로 영상처리 입문에서 가장 먼저 다루는 내용이 화소값 기반 처리이다. 이것은 영상 구조에 대해 알기 위해 가장 먼저 이해해야 하는 것이 화소에 대한 기본 개념이기 때문이다.

디지털 영상은 이 화소들의 집합을 의미하며, 이 화소들에 대해 다양한 연산을 하는 것이 영상처리이다.

## 6.1 영상화소의 접근

영상처리를 아주 간단하게 말해보면, 2차원 데이터에 대한 행렬 연산이라고 할 수 있다.
따라서 영상을 다루려면 기본적으로 영상의 화소에 접근하고, 그 값을 수정하거나 새로 만들 수 있어야 한다.

### 6.1.1 화소(행렬 원소) 접근

다음은 모든 원소를 순회하여 원소값을 2배로 변경하는 예제이다.

`-` 방법1

행렬의 원소를 순회하며 직접 원소값을 가져와서 계산

In [1]:
import numpy as np

def mat_access1(mat):
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            k = mat[i, j]
            mat[i, j] = k * 2

In [3]:
mat1 = np.arange(10).reshape(2,5)
mat1

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [6]:
print('원소 처리 전: \n%s\n' % mat1)
mat_access1(mat1)
print('원소 처리 후: \n%s\n' % mat1)

원소 처리 전: 
[[ 0  2  4  6  8]
 [10 12 14 16 18]]

원소 처리 후: 
[[ 0  4  8 12 16]
 [20 24 28 32 36]]



`-` 방법2

행렬 원소를 순회하며, ndarray 클래스의 내부 메서드인 **`item()`** 함수와 **`itemset()`** 함수로 가져와서 값을 변경

In [8]:
def mat_access2(mat):
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            k = mat.item(i, j)  #
            mat.itemset((i, j), k*2)

In [10]:
mat2 = np.arange(10).reshape(2, 5)
mat2

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [11]:
print('원소 처리 전: \n%s\n' % mat2)
mat_access2(mat2)
print('원소 처리 후: \n%s\n' % mat2)

원소 처리 전: 
[[0 1 2 3 4]
 [5 6 7 8 9]]

원소 처리 후: 
[[ 0  2  4  6  8]
 [10 12 14 16 18]]



### 6.1.2  영상 반전을 수행하는 다양한 방법들

행렬을 처리하여 영상의 반전을 수행하는 다양한 방법들을 함수로 만들고, 각 방법의 수행속도를 계산해보자.

In [14]:
# Mat::ptr()을 통한 행렬 원소 접근

import numpy as np, cv2, time


## 화소 직접접근
def pixel_access1(image):
    image1 = np.zeros(image.shape[:2], image.dtype)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image[i,j]                 # 화소접근
            image1[i, j] = 255 - pixel         # 화소할당
            
    return image1
        

In [15]:
## item() 함수
def pixel_access2(image):                         # item() 함수 접근 방법
    image2 = np.zeros(image.shape[:2], image.dtype)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image.item(i, j)              # 화소접근
            image2.itemset((i, j), 255 - pixel)   # 화소할당
    return image2

In [20]:
## 룩업테이블
def pixel_access3(image):
    lut = [255 - i for i in range(256)]
    lut = np.array(lut, np.uint8)
    image3 = lut[image]
    return image3
    

In [61]:
## openCV
def pixel_access4(image):
    image4 = cv2.subtract(255, image)
    return image4

In [18]:
## ndarray 산술연산
def pixel_access5(image):
    image5 = 255 - image
    return image5

In [57]:
image = cv2.imread('./ghtop_images/chap06_images/bright.jpg', cv2.IMREAD_GRAYSCALE)

In [58]:
image.shape

(450, 360)

In [62]:
## 수행시간 체크 함수
def time_check(func, msg):
    start_time = time.perf_counter()
    ret_img = func(image)
    elapsed = (time.perf_counter() - start_time) * 1000
    print(msg, "수행시간 : %0.2f ms" % elapsed )
    return ret_img

In [63]:
image1 = time_check(pixel_access1, "[방법1] 직접 접근 방식")
image2 = time_check(pixel_access2, "[방법2] item() 접근 방식")
image3 = time_check(pixel_access3, "[방법3] 룩업테이블  방식")
image4 = time_check(pixel_access4, "[방법4] OpenCV 함수 방식")
image5 = time_check(pixel_access5, "[방법5] ndarray 방식")

[방법1] 직접 접근 방식 수행시간 : 550.22 ms
[방법2] item() 접근 방식 수행시간 : 108.73 ms
[방법3] 룩업테이블  방식 수행시간 : 0.92 ms
[방법4] OpenCV 함수 방식 수행시간 : 0.08 ms
[방법5] ndarray 방식 수행시간 : 0.19 ms


실행결과를 보면, OpenCV 또는 ndarray 방식으로 화소에 접근하는 경우 속도가 빠른 것을 확인할 수 있었다.

따라서 화소 직접 접근 방법보다는 OpenCV에서 제공하는 함수들을 조합하거나 ndarray 객체의 원소간 연산으로 구현 내용을 만드는 것이 좋다.

## 6.2 화소 밝기 변환

### 6.2.1 그레이 스케일 (명암도) 영상

일반적으로 이해하는 컬러가 아닌 영상을 우리는 흑백영상이라고 쉽게 부르지만, 엄밀한 의미에서 흑백 영상이라는 것은 검은색과 흰색으로 구성된 영상을 의미하기 때문에 단일채널 영상에 이 이름을 붙이는 것이 맞지 않을 수도 있다.

디지털 영상처리에서 보통 단일채널의 영상을 **그레이 스케일(gray-scale)영상** 혹은 **명암도 영상**이라고 한다. 

- 그레이 스케일 영상

    - 0~255의 값을 가지는 화소들이 모여서 구성된 영상 <br>
    - 0은 검은색, 255는 흰색을 의미
    - 0~255 사이 값들은 진한 회색에서 연한 회색까지를 나타냄

In [65]:
import numpy as np
import cv2

In [86]:
# 명암도 영상 생성
image1 = np.zeros((50,512), np.uint8)        # 50x512 영상 생성
image2 = np.zeros((50,512), np.uint8)

rows, cols  = image1.shape[:2]

for i in range(rows):
    for j in range(cols):
        image1.itemset((i,j), j//2)          # 화소값 점진적 증가
        image2.itemset((i,j), j // 20*10)    # 계단 현상 증가

In [85]:
#hide
print('image1.shape:',image1.shape)
print('image2.shape:',image2.shape) # 0값으로 채워진 50x512 행렬
print("image1's rows: ", rows)
print("image1's cols: ", cols)

image1.shape: (50, 512)
image2.shape: (50, 512)
image1's rows:  50
image1's cols:  512


In [87]:
#collapse-hide
cv2.imshow("image1", image1)
cv2.imshow("image2", image2)
cv2.waitKey(0)

cv2.imwrite('./prac_image/image1_226.png', image1) # 이미지 저장
cv2.imwrite('./prac_image/image2_226.png', image2) # 이미지 저장

cv2.destroyAllWindows()

`-` 실행결과

<center> image1</center>

<img src = "./prac_image/image1_226.png">

- 나눗셈 몫 연산자로 2로 나눈 몫을 저장하는 것은 가로 인덱스의 절반 값으로 j열 원소의 화소값을 설정한 것이다.
따라서 화소값은 왼쪽에서 오른쪽으로 0에서 255의 값까지 **점진적으로 증가**한다.

<center> image2 </center>

<img src = "./prac_image/image2_226.png">

- (j // 20 * 10) 은 몫 연산자로 인해서 계산 값의 소수 부분은 날라간다. 따라서 20화소씩 같은 값을 갖게 되어 **계단 현상**을 나타내며 **증가한다.**

### 6.2.2 영상의 화소 표현

영상파일을 읽어 들여 그 영상의 특정 부분의 화소들을 확인해보자. 영상파일을 행렬에 저장하고, 관심 영역을 지정해서 출력하면 간단히 영상 데이터인 화소들의 값을 출력할 수 있다.

In [94]:
#collapse-output

# 영상 화소값 확인 (pixel_value)
import cv2

image = cv2.imread('./ghtop_images/chap06_images/pixel.jpg', cv2.IMREAD_GRAYSCALE)

(x, y), (w, h) = (180, 37), (15, 10)
roi_img = image[y:y+h, x:x+w]   # 행은 시작 y좌표에서 y+h까지, 열은 시작 x좌표에서 x+w까지

#print("[roi img] =\n", roi_img)

- $(x, y)$는 사각형의 시작좌표
- $(w, h)$는 사각형의 크기
- 즉, 사각형의 시작좌표와 크기로 **관심영역을 지정**한다.

In [91]:
print("[roi_img] =")
for row in roi_img:
    for p in row:
        print("%4d" % p, end="")
print()

[roi_img] =
  56  51  59  66  84 104 154 206 220 208 203 207 205 204 204  75  57  53  53  72  71 100 152 195 214 212 201 209 207 205  88  76  65  53  51  60  73  96 143 200 219 200 206 204 202  91  92  80  63  53  59  59  61  89 144 195 222 205 200 205  89  94  90  82  63  54  51  56  65  92 149 203 223 209 196  89  91  90  89  84  64  54  55  51  56  94 140 208 223 203  91  86  84  85  97  86  72  59  50  53  66  81 148 211 216  92  86  85  88  92  95  88  70  55  53  59  64  89 155 211  88  85  86  90  87  87  89  86  72  56  50  53  59  88 175  87  85  86  88  87  84  86  90  86  70  53  44  51  56 111


In [93]:
cv2.rectangle(image, (x,y,w,h) , 255, 1)     # 관심 영역에 사각형 표시
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.imwrite('./prac_image/image_227.png', image) # 이미지 저장

cv2.destroyAllWindows()

<img src = './prac_image/image_227.png'>

- 실행 결과를 보면, 영상의 우상단에 흰색의 작은 사각형이 그려져 있다. 이 사각형이 관심 영역이며, 이 영역의 화소값과 비교해보자.

In [95]:
print("[roi img] =\n", roi_img)

[roi img] =
 [[ 56  51  59  66  84 104 154 206 220 208 203 207 205 204 204]
 [ 75  57  53  53  72  71 100 152 195 214 212 201 209 207 205]
 [ 88  76  65  53  51  60  73  96 143 200 219 200 206 204 202]
 [ 91  92  80  63  53  59  59  61  89 144 195 222 205 200 205]
 [ 89  94  90  82  63  54  51  56  65  92 149 203 223 209 196]
 [ 89  91  90  89  84  64  54  55  51  56  94 140 208 223 203]
 [ 91  86  84  85  97  86  72  59  50  53  66  81 148 211 216]
 [ 92  86  85  88  92  95  88  70  55  53  59  64  89 155 211]
 [ 88  85  86  90  87  87  89  86  72  56  50  53  59  88 175]
 [ 87  85  86  88  87  84  86  90  86  70  53  44  51  56 111]]


- 관심영역 즉, 흰색 사각형이 그려져 있는 부분을 보면 주대각선 윗 부분은 흰색(밝은색)이고 아랫부분은 진한회색(어두운색)임을 알 수 있다.
- 화소 값을 보면 주대각선 기준 윗부분은 화소값은 대략 $200\sim225$범위의 값을 나타내고, 그 아래부분은 대략 $50\sim80$범위의 값임을 확인
- 즉, 흰색부분은 화소값이 255와 가깝고, 어두운 부분은 0에 가까운 값을 갖는다.

### 6.2.3 영상 밝기의 가감영상

화소값이 영상의 밝기를 나타내기 때문에 이 화소값을 변경하면 영상의 밝기를 바꿀 수 있다.

- 예를 들어 영상의 화소에 특정한 상숫값을 더하면 영상이 밝아지고, 상숫값을 빼면 영상이 어두워진다.
- 또한, 화소가 가질 수 있는 최댓값(예로 255)에서 그 화소의 값을 빼면 반전 영상이 만들어진다.

### 6.2.4 행렬 덧셈 및 곱셈을 이용한 영상 합성

영상에 상수를 더하거나 빼는 연산을 확장하면 두 개의 영상을 더하거나 빼는 연산을 생각해 볼 수 있다. 두 영상을 합하면 영상 합성이 되며, 두 영상을 빼면 차영상(difference image)이 된다.

다음은 알렉산더 대왕 동상 영상($A$)과 사도의 건물 영상($B$), 두 영상을 합성한 영상($A+B$)을 구하는 예제이다.

<img src="./prac_image/problem_box.png">

<img src = "./prac_image/solution_box.png">

#### 행렬 합과 곱 연산을 통한 영상 합성

In [1]:
import numpy as np, cv2

image1 = cv2.imread('./ghtop_images/chap06_images/add1.jpg', cv2.IMREAD_GRAYSCALE)    # 영상 읽기
image2 = cv2.imread('./ghtop_images/chap06_images/add2.jpg', cv2.IMREAD_GRAYSCALE)    

In [2]:
##  영상 합성 방법
alpha, beta = 0.6, 0.7                              # 곱셈 비율

add_img1 = cv2.add(image1, image2)                  # 두 영상 단순 더하기

add_img2 = cv2.add(image1 * alpha, image2 * beta)   # 두 영상 비율에 따른 더하기
add_img2 = np.clip(add_img2, 0, 255).astype('uint8') # saturation 처리

add_img3 = cv2.addWeighted(image1, alpha, image2, beta, 0) # 두 영상 비율에 따른 더하기

titles = ['image1', 'image2','add_img1','add_img2','add_img3'] # 윈도우 이름
for t in titles: cv2.imshow(t, eval(t))  # 영상 표시
    
cv2.waitKey(0)

cv2.destroyAllWindows()

In [3]:
titles = ['image1', 'image2','add_img1','add_img2','add_img3']
eval(titles[1]) 

array([[110, 122, 118, ..., 165, 166, 166],
       [143, 159, 168, ..., 165, 166, 166],
       [115, 117, 140, ..., 165, 166, 166],
       ...,
       [ 32,  41,  45, ...,  34,  32,  30],
       [ 27,  35,  40, ..., 110, 109, 108],
       [ 41,  36,  31, ..., 146, 148, 149]], dtype=uint8)

- 파이썬 내장함수 `eval()`함수를 사용하면 리스트 원소의 문자열을 행렬 변수로 사용하여 행렬을 출력해주며, `cv2.imshow()`에 집어넣어 윈도우에 표시한다.

In [47]:
#hide
import os 
path_save = './prac_image'
os.chdir(path_save)
file_name = []
for i in range(len(titles)):    
    file_name.append(path_save + '/' + titles[i] +'_233.png') #, eval(titles[i]))
print(file_name)
    
os.chdir('../')

['./prac_image/image1_233.png', './prac_image/image2_233.png', './prac_image/add_img1_233.png', './prac_image/add_img2_233.png', './prac_image/add_img3_233.png']


In [46]:
#hide
for i in range(len(file_name)):
    cv2.imwrite(file_name[i], eval(titles[i]))
               
print(os.listdir('./prac_image'))

['add_img1_233.png', 'add_img2_233.png', 'add_img3_233.png', 'blue.png', 'green.png', 'image.png', 'image1.png', 'image1_226.png', 'image1_233.png', 'image2_226.png', 'image2_233.png', 'image_227.png', 'img_basic.png', 'img_gray.png', 'red.png', 'repeat.png', 'trans.png', 'xaxis.png', 'xyaxis.png', 'yaxis.png']


<img src="./prac_image/img1_plus_img2.png">

In [48]:
add_img1 = cv2.add(image1, image2)                  # 두 영상 단순 더하기

<img src="./prac_image/add_img1_233.png">

In [49]:
add_img2 = cv2.add(image1 * alpha, image2 * beta)   # 두 영상 비율에 따른 더하기
add_img2 = np.clip(add_img2, 0, 255).astype('uint8') # saturation 처리

<img src="./prac_image/add_img2_233.png">

In [50]:
add_img3 = cv2.addWeighted(image1, alpha, image2, beta, 0) # 두 영상 비율에 따른 더하기

<img src="./prac_image/add_img3_233.png">